In [3]:
import tensorflow as tf
#import matplotlib.pyplot as plt
import random

/Users/seongjuhyeon/workspace/tensorflowZeroToAll/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
# To solve downloading files.
## https://github.com/tensorflow/tensorflow/issues/10779
## run the /Applications/Python 3.6/Install Certificates.command bash script to install newer certs.
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)
print(mnist.train.num_examples)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
55000


In [6]:
# 0 ~ 9까지의 숫자 갯수
nb_classes = 10

In [7]:
# MNIST Data image of shape = 28 * 28 = 784
X = tf.placeholder(tf.float32, shape=[None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1]) # -1: N개, img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, shape=[None, nb_classes]) # one-hot으로 주어짐.

### L1 Image

In [8]:
# 3x3x1(color: black/white), 32 of filters
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
print(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],
                   strides=[1,2,2,1], padding='SAME')
print(L1)

Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


### L2 Image

In [9]:
# Input Image => shape=(?, 14, 14, 32), 64 of filters
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
print(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1],
                   strides=[1,2,2,1], padding='SAME')
print(L2)
input_shape_cnt = 7 * 7 * 64
L2 = tf.reshape(L2, [-1, input_shape_cnt])

Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


### Fully Connected(FC, Dense) Layer

In [10]:
# 10: 0~9 of numbers
W3 = tf.get_variable("W3", shape=[input_shape_cnt, 10]
                    , initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable(name="b1", shape=[10])
hypothesis = tf.matmul(L2, W3) + b

In [14]:
print(hypothesis)

Tensor("add:0", shape=(?, 10), dtype=float32)


In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

In [16]:
# Minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [17]:
# Correct Prediction
prediction = tf.argmax(hypothesis, 1)
real = tf.argmax(Y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, real), dtype=tf.float32))

In [18]:
# Prepare session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [19]:
# Slice the data, epochs, batch_size
training_epochs = 15
batch_size = 100
total_iterator = int(mnist.train.num_examples / batch_size)

print(mnist.train.num_examples)
print(total_iterator)

55000
550


In [22]:
# Fit the train
for epoch in range(training_epochs):
    avg_cost = 0
    total_iterator = total_iterator
    
    for i in range(total_iterator):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c_val, _ = session.run([cost, train], feed_dict=feed_dict)
        avg_cost = c_val / batch_size
    
    print("Epoch:", "%04d" % (epoch+1), ", Cost:", "{:.5f}".format(avg_cost))

print('Learning Finished!')

print("Accuracy:", session.run(accuracy,
                               feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 , Cost: 0.00055
Epoch: 0002 , Cost: 0.00050
Epoch: 0003 , Cost: 0.00011
Epoch: 0004 , Cost: 0.00001
Epoch: 0005 , Cost: 0.00067
Epoch: 0006 , Cost: 0.00001
Epoch: 0007 , Cost: 0.00003
Epoch: 0008 , Cost: 0.00002
Epoch: 0009 , Cost: 0.00002
Epoch: 0010 , Cost: 0.00001
Epoch: 0011 , Cost: 0.00001
Epoch: 0012 , Cost: 0.00000
Epoch: 0013 , Cost: 0.00013
Epoch: 0014 , Cost: 0.00000
Epoch: 0015 , Cost: 0.00002
Learning Finished!
Accuracy: 0.9881


In [28]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", session.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", session.run(tf.argmax(hypothesis, 1),
                                  feed_dict={X: mnist.test.images[r:r + 1]}))

Label:  [2]
Prediction:  [2]
